In [22]:
import sys
import numpy as np
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))
from instruments import *
from physical_models import *

In [23]:
# define parameters from the paper
# test with 170815
bin = 8
offset_imr = -0.0118
theta_imr = 45
offset_hwp = -0.002
theta_hwp = 78.75
theta_cal = -45
offset_cal = -0.035
wavelength_bins = np.array([1159.5614, 1199.6971, 1241.2219, 1284.184 , 1328.6331, 1374.6208,
                1422.2002, 1471.4264, 1522.3565, 1575.0495, 1629.5663, 1685.9701,
                1744.3261, 1804.7021, 1867.1678, 1931.7956, 1998.6603, 2067.8395,
                2139.4131, 2213.4641, 2290.0781, 2369.3441])
hwp_retardances = HWP_retardance(wavelength_bins)
imr_retardances = IMR_retardance(wavelength_bins)
#T0D0 - ask rebecca about imperfect diattenuation of cal


In [20]:
# create a system dict for CHARIS
sys_dict = {
    "components" : {
        # "wollaston" : {
        #     "type" : "wollaston_prism_function",
        #     "properties" : {"beam": "o"},
        #     "tag": "internal",
        # },
        "rot_imr_1" : {
            "type" : "rotator_function",
            "properties" : {"pa": - offset_imr - theta_imr},
            "tag": "internal",
        },
        "image_rotator" : {
            "type" : "Retarder",
            "properties" : {"phi": imr_retardances[bin]},
            "tag": "internal",
        },
        "rot_imr_2" : {
            "type" : "Rotator",
            "properties" : {"pa": offset_imr + theta_imr},
            "tag": "internal",
        },
        "rot_hwp_1" : {
            "type" : "Rotator",
            "properties" : {"pa": - offset_hwp - theta_hwp},
            "tag": "internal",
        },
        "hwp" : {
            "type" : "Retarder",
            "properties" : {"phi": hwp_retardances[bin]},
            "tag": "internal",
        },
        "rot_hwp_2" : {
            "type": "Rotator",
            "properties": {"pa": offset_hwp + theta_hwp},
            "tag": "internal",
        },
        "rot_cal" : {
            "type": "Rotator",
            "properties": {"pa": offset_cal + theta_cal},
            "tag": "internal",
        },
        "calibration_polarizer" : {
            "type": "Polarizer",
            "properties": {"p_x": 1, "p_y": 0},
            "tag": "internal",
        },
        "order" : [
            "calibration_polarizer", "rot_cal", "rot_hwp_2", "hwp", "rot_hwp_1", 
            "rot_imr_2", "image_rotator", "rot_imr_1", "wollaston"
        ]
    }
}

In [21]:
system_mm = generate_system_mueller_matrix(sys_dict)
print(system_mm.evaluate())

AttributeError: module 'inspect' has no attribute 'getargspec'